In [1]:
pip install nba_api

Note: you may need to restart the kernel to use updated packages.


In [2]:
import nba_api
from nba_api.stats.endpoints import shotchartdetail
import pandas as pd
import json
import requests
import datetime
import numpy as np
import time

In [3]:
def pull_season(season, type = 'Regular Season'):
    
    chart = shotchartdetail.ShotChartDetail(team_id=0, player_id=0,
                                            context_measure_simple = 'FGA',
                                            season_nullable=season,
                                            season_type_all_star=type)
    chart = chart.get_json()

    #convert to dict
    chart_dict = json.loads(chart)

    #pull the results from dictionary
    chart_data = chart_dict['resultSets'][0]

    #headers and rows
    headers = chart_data['headers']
    rows = chart_data['rowSet']

    # convert to df
    chart_df = pd.DataFrame(rows,columns = headers)

    return chart_df

In [4]:
seasons = ['2000-01','2001-02','2002-03','2003-04','2004-05',
           '2005-06','2006-07','2007-08','2008-09','2009-10',
           '2010-11','2011-12','2012-13','2013-14','2014-15',
           '2015-16','2016-17','2017-18','2018-19','2019-20',
           '2020-21','2021-22','2022-23']

In [7]:
df = []

for season in seasons:
    df.append(pull_season(season))
    time.sleep(5)

In [8]:
shot_chart = pd.concat(df, ignore_index=True)

In [9]:
pd.set_option('display.max_columns',None)
shot_chart.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0020000001,2,84,Latrell Sprewell,1610612752,New York Knicks,1,11,41,Missed Shot,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,-58,28,1,0,20001031,NYK,PHI
1,Shot Chart Detail,0020000001,11,947,Allen Iverson,1610612755,Philadelphia 76ers,1,11,8,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,19,-107,167,1,0,20001031,NYK,PHI
2,Shot Chart Detail,0020000001,13,275,Allan Houston,1610612752,New York Knicks,1,10,53,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,15,-151,13,1,0,20001031,NYK,PHI
3,Shot Chart Detail,0020000001,16,238,Tyrone Hill,1610612755,Philadelphia 76ers,1,10,35,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,9,-94,17,1,0,20001031,NYK,PHI
4,Shot Chart Detail,0020000001,18,84,Latrell Sprewell,1610612752,New York Knicks,1,10,19,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,13,-101,95,1,0,20001031,NYK,PHI


In [10]:
shot_chart['HTM'].value_counts()

HTM
GSW    158906
PHX    158242
LAL    157275
SAC    156860
DEN    156671
HOU    154911
MIN    154786
WAS    154419
MIL    154375
IND    154368
DAL    154314
CHI    154294
ATL    154060
CLE    153677
ORL    153580
TOR    153383
POR    153369
LAC    153045
PHI    152804
SAS    152797
BOS    152267
NYK    152226
DET    151168
MIA    148659
UTA    148453
MEM    146379
CHA    126501
OKC    102101
NJN     76802
BKN     76069
NOP     69895
NOH     57261
SEA     52904
CHH     13074
NOK     13046
VAN      6642
Name: count, dtype: int64

In [11]:
shot_chart['TEAM_NAME'].value_counts()

TEAM_NAME
Golden State Warriors                158705
Phoenix Suns                         157668
Sacramento Kings                     156505
Denver Nuggets                       156320
Los Angeles Lakers                   156129
Milwaukee Bucks                      155581
Minnesota Timberwolves               155425
Washington Wizards                   154982
Dallas Mavericks                     154831
Toronto Raptors                      154456
Chicago Bulls                        154190
Indiana Pacers                       153732
Portland Trail Blazers               153632
Orlando Magic                        153337
New York Knicks                      153282
Philadelphia 76ers                   153176
Houston Rockets                      153156
Atlanta Hawks                        152920
Detroit Pistons                      152574
San Antonio Spurs                    152509
Cleveland Cavaliers                  152297
Boston Celtics                       152174
Utah Jazz             

In [12]:
team_corrections = {'NOK':'NOP', 'VAN':'MEM', 'CHH':'CHA', 'NOH': 'NOP', 'SEA':'OKC','NJN': 'BKN'}

In [13]:
shot_chart['HTM'] = shot_chart['HTM'].replace(team_corrections)
shot_chart['VTM'] = shot_chart['VTM'].replace(team_corrections)

In [14]:
team_abbr = {
'Atlanta Hawks':'ATL', 
'Boston Celtics':'BOS', 
'Brooklyn Nets':'BKN', 
'Charlotte Bobcats':'CHA',  
'Charlotte Hornets':'CHA', 
'Chicago Bulls':'CHI',
'Cleveland Cavaliers':'CLE', 
'Dallas Mavericks':'DAL', 
'Denver Nuggets':'DEN',
'Detroit Pistons':'DET', 
'Golden State Warriors':'GSW', 
'Houston Rockets':'HOU',
'Indiana Pacers':'IND', 
'LA Clippers':'LAC', 
'Los Angeles Clippers':'LAC', 
'Los Angeles Lakers':'LAL', 
'Memphis Grizzlies':'MEM',  
'Miami Heat':'MIA',
'Milwaukee Bucks':'MIL', 
'Minnesota Timberwolves':'MIN', 
'New Jersey Nets':'BKN', 
'New Orleans Hornets':'NOP',  
'New Orleans Pelicans':'NOP', 
'New Orleans/Oklahoma City Hornets':'NOP',
'New York Knicks':'NYK',
'Oklahoma City Thunder':'OKC', 
'Orlando Magic':'ORL', 
'Philadelphia 76ers':'PHI',
'Phoenix Suns':'PHX', 
'Portland Trail Blazers':'POR', 
'Sacramento Kings':'SAC',
'San Antonio Spurs':'SAS', 
'Seattle SuperSonics':'OKC', 
'Toronto Raptors':'TOR',
'Utah Jazz':'UTA', 
'Vancouver Grizzlies':'MEM', 
'Washington Bullets':'WAS',  
'Washington Wizards':'WAS'
}

In [15]:
shot_chart['TEAM_NAME_NEW'] = shot_chart['TEAM_NAME'].map(team_abbr)

In [16]:
#New column to determine who the opposing team was
shot_chart['OPPONENT'] = np.where(shot_chart['TEAM_NAME_NEW'] == shot_chart['HTM'], shot_chart['VTM'], shot_chart['HTM'])

#New column to determine who the home/away team was for each player
shot_chart['HOME_AWAY'] = np.where(shot_chart['TEAM_NAME_NEW'] == shot_chart['HTM'], 'Home', 'Away')

#New column fixing date format
shot_chart['GAME_DATE_FIXED'] = pd.to_datetime(shot_chart['GAME_DATE'], format='%Y%m%d').dt.strftime('%m-%d-%Y')
shot_chart['GAME_DATE_FIXED'] = pd.to_datetime(shot_chart['GAME_DATE_FIXED'])

In [17]:
seasons_dates = {
    (pd.to_datetime('10-01-2000'), pd.to_datetime('09-30-2001')): '2000-2001',
    (pd.to_datetime('10-01-2001'), pd.to_datetime('09-30-2002')): '2001-2002',
    (pd.to_datetime('10-01-2002'), pd.to_datetime('09-30-2003')): '2002-2003',
    (pd.to_datetime('10-01-2003'), pd.to_datetime('09-30-2004')): '2003-2004',
    (pd.to_datetime('10-01-2004'), pd.to_datetime('09-30-2005')): '2004-2005',
    (pd.to_datetime('10-01-2005'), pd.to_datetime('09-30-2006')): '2005-2006',
    (pd.to_datetime('10-01-2006'), pd.to_datetime('09-30-2007')): '2006-2007',
    (pd.to_datetime('10-01-2007'), pd.to_datetime('09-30-2008')): '2007-2008',
    (pd.to_datetime('10-01-2008'), pd.to_datetime('09-30-2009')): '2008-2009',
    (pd.to_datetime('10-01-2009'), pd.to_datetime('09-30-2010')): '2009-2010',
    (pd.to_datetime('10-01-2010'), pd.to_datetime('09-30-2011')): '2010-2011',
    (pd.to_datetime('10-01-2011'), pd.to_datetime('09-30-2012')): '2011-2012',
    (pd.to_datetime('10-01-2012'), pd.to_datetime('09-30-2013')): '2012-2013',
    (pd.to_datetime('10-01-2013'), pd.to_datetime('09-30-2014')): '2013-2014',
    (pd.to_datetime('10-01-2014'), pd.to_datetime('09-30-2015')): '2014-2015',
    (pd.to_datetime('10-01-2015'), pd.to_datetime('09-30-2016')): '2015-2016',
    (pd.to_datetime('10-01-2016'), pd.to_datetime('09-30-2017')): '2016-2017',
    (pd.to_datetime('10-01-2017'), pd.to_datetime('09-30-2018')): '2017-2018',
    (pd.to_datetime('10-01-2018'), pd.to_datetime('09-30-2019')): '2018-2019',
    (pd.to_datetime('10-01-2019'), pd.to_datetime('09-30-2020')): '2019-2020',
    (pd.to_datetime('10-01-2020'), pd.to_datetime('09-30-2021')): '2020-2021',
    (pd.to_datetime('10-01-2021'), pd.to_datetime('09-30-2022')): '2021-2022',
    (pd.to_datetime('10-01-2022'), pd.to_datetime('09-30-2023')): '2022-2023'
}

In [18]:
#Function to map date ranges to seasons
def map_season(date):
    for interval, season in seasons_dates.items():
        if interval[0] <= date <= interval[1]:
            return season
    return None

#Apply the map_season function to create the 'SEASON' column
shot_chart['SEASON'] = shot_chart['GAME_DATE_FIXED'].apply(map_season)

In [19]:
shot_chart['TIME_OF_SEASON'] = 'Regular Season'

In [20]:
shot_chart.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,TEAM_NAME_NEW,OPPONENT,HOME_AWAY,GAME_DATE_FIXED,SEASON,TIME_OF_SEASON
0,Shot Chart Detail,0020000001,2,84,Latrell Sprewell,1610612752,New York Knicks,1,11,41,Missed Shot,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,-58,28,1,0,20001031,NYK,PHI,NYK,PHI,Home,2000-10-31,2000-2001,Regular Season
1,Shot Chart Detail,0020000001,11,947,Allen Iverson,1610612755,Philadelphia 76ers,1,11,8,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,19,-107,167,1,0,20001031,NYK,PHI,PHI,NYK,Away,2000-10-31,2000-2001,Regular Season
2,Shot Chart Detail,0020000001,13,275,Allan Houston,1610612752,New York Knicks,1,10,53,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,15,-151,13,1,0,20001031,NYK,PHI,NYK,PHI,Home,2000-10-31,2000-2001,Regular Season
3,Shot Chart Detail,0020000001,16,238,Tyrone Hill,1610612755,Philadelphia 76ers,1,10,35,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,9,-94,17,1,0,20001031,NYK,PHI,PHI,NYK,Away,2000-10-31,2000-2001,Regular Season
4,Shot Chart Detail,0020000001,18,84,Latrell Sprewell,1610612752,New York Knicks,1,10,19,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,13,-101,95,1,0,20001031,NYK,PHI,NYK,PHI,Home,2000-10-31,2000-2001,Regular Season


Playoffs

In [21]:
df_playoffs = []

for season in seasons:
    df_playoffs.append(pull_season(season,'Playoffs'))
    time.sleep(3)

In [22]:
shot_chart_playoffs = pd.concat(df_playoffs, ignore_index=True)

In [23]:
shot_chart_playoffs['HTM'] = shot_chart_playoffs['HTM'].replace(team_corrections)
shot_chart_playoffs['VTM'] = shot_chart_playoffs['VTM'].replace(team_corrections)

In [24]:
shot_chart_playoffs['TEAM_NAME_NEW'] = shot_chart_playoffs['TEAM_NAME'].map(team_abbr)

In [25]:
#New column to determine who the opposing team was
shot_chart_playoffs['OPPONENT'] = np.where(shot_chart_playoffs['TEAM_NAME_NEW'] == shot_chart_playoffs['HTM'], shot_chart_playoffs['VTM'], shot_chart_playoffs['HTM'])

#New column to determine who the home/away team was for each player
shot_chart_playoffs['HOME_AWAY'] = np.where(shot_chart_playoffs['TEAM_NAME_NEW'] == shot_chart_playoffs['HTM'], 'Home', 'Away')

#New column fixing date format
shot_chart_playoffs['GAME_DATE_FIXED'] = pd.to_datetime(shot_chart_playoffs['GAME_DATE'], format='%Y%m%d').dt.strftime('%m-%d-%Y')
shot_chart_playoffs['GAME_DATE_FIXED'] = pd.to_datetime(shot_chart_playoffs['GAME_DATE_FIXED'])

In [26]:
shot_chart_playoffs['TIME_OF_SEASON'] = 'Playoffs'

In [27]:
#Apply the map_season function to create the 'SEASON' column
shot_chart_playoffs['SEASON'] = shot_chart_playoffs['GAME_DATE_FIXED'].apply(map_season)

Combine regular and playoffs

In [28]:
full_shot_chart = pd.concat([shot_chart,shot_chart_playoffs])

In [29]:
full_shot_chart.to_csv('nba_shot_chart_full2.csv',index=False)